# __Imports__

In [ ]:
# [+] Libraries that are commented out are only for ROS
#import rospy
import numpy as np
import cv2
#from std_msgs.msg import Float64
#from sensor_msgs.msg import Image
#from cv_bridge import CvBridge, CvBridgeError
import math
import time

#[+------:
import sys
import numpy as np

# __Gate__

In [27]:
current_yaw = 0
cv_image = 0
YAW_VARIANCE = .017

# [+]- Access the camera, the int parameter determines which camera you are using, may have to change in depending on what computer you run on
video_capture = cv2.VideoCapture(0)
# video_capture = cv2.VideoCapture(1)
# video_capture = cv2.VideoCapture(2)

#[+]- Each iteration of this loop processes a frame that is captured by the camera by applying a series of filters. Each filter is in intermediate step, the final image is the one which we annotate and extract information from. 
while(True):
    # Capture the frames
    ret, frame = video_capture.read()
    final = frame

    # change from bgr values to hsv values
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # color bounds of color to be filtered out [+] correction: colors to be NOT filtered out
    # lower_color_bounds = np.array([0, 200, 0])
    # upper_color_bounds = np.array([255, 255, 255])
    lower_color_bounds = np.array([0, 100, 20])
    upper_color_bounds = np.array([30, 255, 255])


    # [+]- Filter 1: threshold shows in black the pixels being filtered out [+: show only the colors that are between the two bounds)
    threshold = cv2.inRange(hsv, lower_color_bounds, upper_color_bounds)

    #[+]- Filter 2:  erode to remove noise
    kernel = np.ones((10, 10), np.uint8)
    erode = cv2.erode(threshold, kernel)

    # get the contours, Each individual contour is a Numpy array of (x,y) coordinates of boundary points of the object
    contours = cv2.findContours(erode, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)[0]

    # create bounding boxes, creates an array of contour areas so that we can know which is th ebiggest
    contour_areas = [cv2.contourArea(x) for x in contours]  
    
    # get the areas of each contour, returns a list of indeces for contour areas array
    contour_indexes = np.argsort(contour_areas)  
    
    # sort the indexes of the largest areas
    for i in contour_indexes[-2:]:  # only look at the two largest contours
        (x,y,w,h) = cv2.boundingRect(contours[i])  # get the location/dimensions of a bounding box for the contour: x,y=coordinates, w,h=dims
        cv2.rectangle(final, pt1=(x,y), pt2=(x+w,y+h), color=(255,0,0), thickness=5)  # draw the bounding box on the image

        # for visibility, we will place a background fill on the contour label
        text = "gatepost"
        text_size, _ = cv2.getTextSize(text, cv2.FONT_HERSHEY_PLAIN, 1, 1)
        text_w, text_h = text_size
        cv2.rectangle(final, pt1=(x, y), pt2=(x + text_w, y - 2*text_h), color=(255, 0, 0), thickness=-1)
        cv2.putText(final, "gatepost", org=(x, y-5), fontFace=cv2.FONT_HERSHEY_PLAIN, fontScale=1, color=(255, 255, 255), thickness=1)

    # [+]- Show each filter that is applied to the frame. Each filter is shown in a separate window
    # cv2.imshow('1 Gate: original frame',frame)
    cv2.imshow('2 Gate: threshold', threshold)
    cv2.imshow("3 Gate: eroded", erode)
    cv2.imshow("4 Gate: final", final)


    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


# __Buoy__ 

In [75]:
current_yaw = 0
cv_image = 0
YAW_VARIANCE = .017

# [+]- Access the camera, the int parameter determines which camera you are using, may have to change in depending on what computer you run on
video_capture = cv2.VideoCapture(0)
# video_capture = cv2.VideoCapture(1)
# video_capture = cv2.VideoCapture(2)

# # color bounds of color to be [NOT] filtered out, by default it filters out light colors so that it can identif 
lower_value_bounds = np.array([0, 0, 0])
upper_value_bounds = np.array([255, 255, 45])


#[+:]-- -------------------------------------------------//
lower_value_bounds2 = np.array([0, 0, 0])
upper_value_bounds2 = np.array([255, 255, 80])
clk= 0
clkRate= 9
clkLim= clkRate*255
#-------------------------------------------------------//

dilateKernel = np.ones((5, 5), np.uint8)
erodeKernel = np.ones((10, 10), np.uint8)
erodeKernel2= np.ones((6, 6), np.uint8)
blurnel = (5, 5)


while(True):
    # Capture the frames
    ret, img_frame = video_capture.read()
    img_final = img_frame

    # change from bgr values to hsv values
    img_hsv = cv2.cvtColor(img_frame, cv2.COLOR_BGR2HSV)

    # threshold shows in black the pixels being filtered out
    img_threshold = cv2.inRange(img_hsv, lower_value_bounds, upper_value_bounds)

    #  [+:] -- Use this to find the ideal value range, which changes depending on the lighting ----------//
    tick= clk//clkRate
    # img_threshold = cv2.inRange(img_hsv, np.array([0, 0, 0]), np.array([255, 255, tick]))
    # cv2.putText(img_final, "[:+:] CLK: "+ str(tick), org=(10, 20), fontFace=cv2.FONT_HERSHEY_DUPLEX, fontScale=.7, color=(200, 150, 0), thickness=2)
    # clk+= 1
    # if clk>clkLim: clk=0
    #-----------------------------------------------------------------------------------------------------//

    # dilate to combine contours
    img_dilate = cv2.dilate(img_threshold, dilateKernel, iterations=3)

    # erode to remove noise
    img_erode = cv2.erode(img_dilate, erodeKernel, iterations=2)

    # blur to smooth edges so circle detection is easier
    img_blur = cv2.blur(img_erode, ksize= blurnel)

    # detect circles
    circles = cv2.HoughCircles(img_blur, cv2.HOUGH_GRADIENT, 1, 20,
                  param1=30,  # edge detection parameter
                  param2=30,  # accumulator threshold, or how circley the circles need to be to be recognized (higher=more circlely)
                  minRadius=0,
                  maxRadius=100)

    # if a circle is identified
    if (type(circles)) is np.ndarray:
        # draw the circles
        circle_radii = [x[2] for x in circles[0]]  # get the radii of each contour
        circle_indexes = np.argsort(circle_radii)  # sort by largest radius
        for i in circle_indexes[-2:]:  # only contour at the largest circles
            circle = circles[0][i]  # get the largest circle
            cv2.circle(img_final, center=(int(circle[0]), int(circle[1])), radius=int(circle[2]), color=(100, 0, 255), thickness=2)  # draw the circle on the image
            # make the text centered
            # text = "police buoy"
            text = " --  -- "
            text_size, _ = cv2.getTextSize(text, cv2.FONT_HERSHEY_PLAIN, 1, 1)  # get the text size
            text_w, text_h = text_size  # get the text width/height
            cv2.putText(img_final, text, org=(int(circle[0])-text_w, int(circle[1])+(text_h//2)+8), fontFace=cv2.FONT_HERSHEY_PLAIN, fontScale=2, color=(100, 0, 255), thickness=2)









    #[:+:]===============================================================================================================================================[:+:]
    clkRate= 1
    #[+]-------- ----------------------------------------------------------------------------//
    img_threshold2 = cv2.inRange(img_hsv, np.array([0, 0, 0]), np.array([255, 255, 80]))
    cv2.putText(img_final, ("[:+:] CLK: "+ str(tick)), org=(10, 20), fontFace=cv2.FONT_HERSHEY_DUPLEX, fontScale=.7, color=(100, 255, 100), thickness=2)
    clk+= 1
    if clk>clkLim: clk=0
    #----------------------------------------------------------------------------------------//

    img_erode2 = cv2.erode(img_threshold2, erodeKernel2, iterations=1)

    dilateKernel2 = np.ones((5, 5), np.uint8)

    img_dilate2 = cv2.dilate(img_erode2, dilateKernel2, iterations=3)

    img_blur2 = cv2.blur(img_erode2, ksize=blurnel)

    circles2 = cv2.HoughCircles(img_blur2, cv2.HOUGH_GRADIENT, 1, 20,
                  param1=30,  # edge detection parameter
                  param2=30,  # accumulator threshold, or how circley the circles need to be to be recognized (higher=more circlely)
                  minRadius=0,
                  maxRadius=100)

    #[+:] --- Circle Array # 2 ----------
    if (type(circles2)) is np.ndarray:
        circle_radii = [x[2] for x in circles2[0]]  
        circle_indexes = np.argsort(circle_radii)  
        for i in circle_indexes[-2:]:  
            circle = circles2[0][i] 
            cv2.circle(img_final, center=(int(circle[0]), int(circle[1])), radius=int(circle[2])+25, color=(200, 150, 0), thickness=2) 
            text = "[:+:]"
            text_size, _ = cv2.getTextSize(text, cv2.FONT_HERSHEY_DUPLEX, 1, 1)  
            text_w, text_h = text_size  
            cv2.putText(img_final, text, org=(int(circle[0])-text_w, int(circle[1])+text_h), fontFace=cv2.FONT_HERSHEY_DUPLEX, fontScale=2, color=(200, 150, 0), thickness=2)
    #[:\:]===============================================================================================================================================[:\:]



    # [+] show frames
    # cv2.imshow('1 original frame', img_frame)
    # cv2.imshow('2 Buoy: hsv', img_hsv)
    # cv2.imshow('3 Buoy: threshold', img_threshold)
    # cv2.imshow("4 Buoy: dilated", img_dilate)
    # cv2.imshow("5 Buoy: eroded", img_erode)
    # cv2.imshow("6 Buoy: blur", img_blur)
    # cv2.imshow("7 Buoy: final", img_final)

    #[+: ------------------------------------------------- ::
    # cv2.imshow('1 original frame2', img_frame)
    # cv2.imshow('2 Buoy: hsv2', img_hsv)
    cv2.imshow('3 Buoy: threshold2', img_threshold2)
    cv2.imshow("4 Buoy: dilated2", img_dilate2)
    cv2.imshow("5 Buoy: eroded2", img_erode2)
    # cv2.imshow("6 Buoy: blur2", img_blur2)
    #::-----------------------------------------------------:+]
    cv2.imshow("7 Buoy: final2", img_final)

  
  

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cv2.destroyAllWindows()


##  __[:+:] Text Detection__

In [2]:



current_yaw = 0
cv_image = 0
YAW_VARIANCE = .017

# [+]- Access the camera, the int parameter determines which camera you are using, may have to change in depending on what computer you run on
video_capture = cv2.VideoCapture(0)
# video_capture = cv2.VideoCapture(1)
# video_capture = cv2.VideoCapture(2)

while(True):
    ret, frame = video_capture.read()
    final = frame

    #--option 1: use grayscale-----
    img2gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    ret, grayMask = cv2.threshold(img2gray, 180, 255, cv2.THRESH_BINARY)
    imgFinal_gray = cv2.bitwise_and(img2gray, img2gray, mask=grayMask)

    #---Option 2: use hsv---
    img2hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    lower = np.array([0, 0, 218])
    upper = np.array([157, 54, 255])
    hsvMask = cv2.inRange(img2hsv, lower, upper)
    # imgFinal_hsv = 



    # for black text , cv.THRESH_BINARY_INV
    #ret, new_img = cv2.threshold(imgFinal_gray, 180, 255, cv2.THRESH_BINARY)  
    ret, new_img = cv2.threshold(imgFinal_gray, 180, 255, cv2.THRESH_BINARY_INV)  



    # to manipulate the orientation of dilution , large x means horizonatally dilating  more, large y means vertically dilating more
    kernel = cv2.getStructuringElement(cv2.MORPH_CROSS, (3,3))  

    # dilate , more the iteration more the dilation
    dilated = cv2.dilate(new_img, kernel, iterations=9)  


    # for cv2.x.x: 
    # _, contours, hierarchy = cv2.findContours(dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)  # findContours returns 3 variables for getting contours

    # for cv3.x.x  comment above line and uncomment line below:
    #image, contours, hierarchy = cv2.findContours(dilated,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)

    # [::]-- Selection: 
    contours, hierarchy = cv2.findContours(dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    for contour in contours:
        # get rectangle bounding contour
        [x, y, w, h] = cv2.boundingRect(contour)

        # Don't plot small false positives that aren't text
        if w < 35 and h < 35:
            continue

        # draw rectangle around contour on original image
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 255), 2)

 
    #cv2.imshow('original frame',frame)
    #cv2.imshow('threshold', imgFinal_gray)
    #cv2.imshow("dilated", dilated)
    cv2.imshow("grayMask", grayMask)
    #cv2.imshow("hsvMask", hsvMask)
    cv2.imshow("new", new_img)
    cv2.imshow("final", final)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cv2.destroyAllWindows()